<a href="https://colab.research.google.com/github/abhiverma1924/Machine-Learning./blob/master/Gender_predictionByName.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Dense, LSTM ,Dropout
from sklearn.model_selection import train_test_split
from keras.models import Sequential

In [0]:
female =pd.read_csv("Female-Names.csv")

In [0]:
male = pd.read_csv("Male-Names.csv")

In [61]:
female.tail()

,name,gender,race
15377,saroj devi,f,indian
15378,naina @ geeta,f,indian
15379,manju d/0 baboo lal jatav,f,indian
15380,shivani,f,indian
15381,nayna,f,indian


In [62]:
male.tail()

,name,gender,race
14840,buddha,m,indian
14841,mukesh,m,indian
14842,monu,m,indian
14843,govind prasad shahu,m,indian
14844,deepak,m,indian


In [0]:
gender = pd.concat([female,male])

In [64]:
gender.head()

,name,gender,race
0,shivani,f,indian
1,isha,f,indian
2,smt shyani devi,f,indian
3,divya,f,indian
4,mansi,f,indian


In [65]:
gender.shape

(30227, 3)

In [0]:
char = [' ', '.', 'END', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']    

In [0]:
charIndex ={}
index =0
for ch in char:
  charIndex[ch] = index
  index+=1
  

In [0]:
def keep(word):
  if len(word)>19:
    return False
  for ch in word:
    if ch not in set(char):
      return False
  return True

In [0]:
def clean(word):
  word = str(word)
  char = [' ', '.','END', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']  
  name = word.lower()
  if keep(name):
    return name
  else:
    return None

In [99]:
gender.name = gender.name.apply(lambda x: clean(x))

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [0]:
#gender

In [0]:
gender = gender.dropna()

In [0]:
#gender

In [0]:
def convert_one_hot(word):
  word_char=[]
  max_len = 20 
  for ch in word:
    one_hot_vec = np.zeros(len(char),)
    one_hot_vec[charIndex[ch]] = 1
    word_char.append(one_hot_vec)
  for i in range(max_len-len(word)):
    one_hot_vec = np.zeros(len(char),)
    one_hot_vec[charIndex['END']]=1
    word_char.append(one_hot_vec)
  
  
  word_char = np.array(word_char)
  return word_char
    

In [0]:
word = []
label =[]

for n , g in gender.drop('race',axis=1).itertuples(index=False):
  word.append(convert_one_hot(n))
  label.append(g)

label = np.array(label) 
word = np.array(word)

In [0]:
word[0]
#label.shape

In [0]:
ohe = OneHotEncoder()

In [0]:
y = ohe.fit_transform(label.reshape(-1,1)).todense()

In [125]:
y.shape

(27980, 2)

In [139]:
model = Sequential()
model.add(LSTM(64,input_shape=(20,29), return_sequences= True))
model.add(LSTM(64))
model.add(Dropout(.3))
model.add(Dense(2, activation="sigmoid"))
model.summary()

W0722 12:35:32.172215 140355273648000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0722 12:35:32.181618 140355273648000 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 20, 64)            24064     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 57,218
Trainable params: 57,218
Non-trainable params: 0
_________________________________________________________________


In [0]:
 X_train, X_test, y_train, y_test = train_test_split(
...     word, y, test_size=0.33, random_state=42)

In [146]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop", metrics= ["accuracy"])

W0722 12:39:31.491922 140355273648000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0722 12:39:31.519475 140355273648000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0722 12:39:31.525366 140355273648000 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [147]:
model.fit(X_train,y_train,epochs= 10)

Epoch 1/10
18746/18746 [==============================] - 39s 2ms/step - loss: 0.5395 - acc: 0.7265
Epoch 2/10
18746/18746 [==============================] - 34s 2ms/step - loss: 0.4075 - acc: 0.8236
Epoch 3/10
18746/18746 [==============================] - 34s 2ms/step - loss: 0.3477 - acc: 0.8600
Epoch 4/10
18746/18746 [==============================] - 34s 2ms/step - loss: 0.3090 - acc: 0.8794
Epoch 5/10
18746/18746 [==============================] - 34s 2ms/step - loss: 0.2813 - acc: 0.8942
Epoch 6/10
18746/18746 [==============================] - 34s 2ms/step - loss: 0.2629 - acc: 0.9037
Epoch 7/10
18746/18746 [==============================] - 34s 2ms/step - loss: 0.2456 - acc: 0.9114
Epoch 8/10
18746/18746 [==============================] - 34s 2ms/step - loss: 0.2340 - acc: 0.9175
Epoch 9/10
18746/18746 [==============================] - 34s 2ms/step - loss: 0.2207 - acc: 0.9235
Epoch 10/10
18746/18746 [==============================] - 34s 2ms/step - loss: 0.2126 - acc: 0.9278

In [148]:
model.evaluate(X_test,y_test)

9234/9234 [==============================] - 7s 726us/step


[0.23272504039933367, 0.9179120640592552]

In [150]:
name = "neha"
model.predict(convert_one_hot(name).reshape(1,20,29))

array([[0.9728688 , 0.02673032]], dtype=float32)